In [1]:
def bfs(graph, s):
    path_found = []
    couples_found = []
    
    visited = {k: False for k in graph.keys()}
    queue = []
    
    queue.append(s)
    visited[s] = True
    
    while queue:
        s = queue.pop(0)
        path_found.append(s)

        for i in graph[s]:
            if visited[i] == False:
                queue.append(i)
                visited[i] = True
                couples_found.append([s, i])
    
    return path_found, couples_found

In [2]:
def start_flow(flow_network, base_value=0):
    flow = {k: [] for k in flow_network.keys()}
    
    for k in flow_network.keys():
        for p in flow_network[k]:
            flow[k].append([p[0], base_value, p[1]])
            
    return flow

In [3]:
def residual_network(flow):
    residual_graph = {n:[] for n in flow.keys()}

    for n in flow.keys():
        nn = flow[n].copy()

        for node in [v[0] for v in nn]:
            p_node = [p for p in flow[n] if p[0] == node]
            
            act_use = p_node[0][1]
            act_cap = p_node[0][2]
            f_dir = act_cap-act_use
            
            if f_dir != 0:
                residual_graph[n].append([node, f_dir])
            if act_use != 0:
                residual_graph[node].append([n, act_use])
            
    return residual_graph

In [4]:
def min_cf_g(couples_path, residual_network):
    min_cf = None

    for c in couples_path:
        act_cf = [p for p in residual_network[c[0]] if p[0] == c[1]][0][1]

        if min_cf == None:
            min_cf = act_cf
        elif min_cf > act_cf:
            min_cf = act_cf

    return min_cf

In [5]:
def update_f_value_sum(flow, u, v, f_value):
    [p for p in flow[u] if p[0] == v][0][1] = [p for p in flow[u] if p[0] == v][0][1] + f_value
    
    return flow

In [6]:
def path_to_t_bfs(couples, t="T"):
    fpath_to_t = []
    act_coup = [p for p in couples if t in p]
    if len(act_coup) == 0:
        return False
    act_coup = act_coup[0]
    fpath_to_t.insert(0, act_coup)
    
    match_par = [c for c in couples if c[1] == act_coup[0]]
    
    if len(match_par) > 1:
        assert False, "Caso não previsto, filho com mais de 1 pai"

    while len(match_par) != 0:
        act_coup = match_par[0]
        fpath_to_t.insert(0, act_coup)
        
        match_par = [c for c in couples if c[1] == act_coup[0]]
        if len(match_par) > 1:
            assert False, "Caso não previsto, filho com mais de 1 pai"
    
    return fpath_to_t

In [7]:
def ford_fulkerson_algorithm(flow_network):
    flow = start_flow(flow_network)
    res_network = residual_network(flow)
        
    res_network_bfs = dict()

    for k in res_network.keys():
        res_network_bfs[k] = [p[0] for p in res_network[k]]
        
    path_to_t, couples_to_t = bfs(res_network_bfs, "S")
    couples_to_t = path_to_t_bfs(couples_to_t)
    
    print("f:", flow)
    print("r:", res_network)
    print("p:", couples_to_t)
    print()
    
    while couples_to_t != False:
        min_cf_l = min_cf_g(couples_to_t, res_network)

        for p in couples_to_t:
            u = p[0]
            v = p[1]

            exist_p = [n for n in flow[u] if n[0] == v]

            if len(exist_p) > 0:
                flow = update_f_value_sum(flow, u, v, min_cf_l)
            else:
                flow = update_f_value_sum(flow, v, u, -min_cf_l)

        res_network = residual_network(flow)

        res_network_bfs = dict()

        for k in res_network.keys():
            res_network_bfs[k] = [p[0] for p in res_network[k]]

        path_to_t, couples_to_t = bfs(res_network_bfs, "S")
        couples_to_t = path_to_t_bfs(couples_to_t)
        
        print("f:", flow)
        print("r:", res_network)
        print("p:", couples_to_t)
        print()
    
    return flow

In [8]:
flow_network_p2 = {
    "S": [["A", 20], ["B", 30], ["C", 20]],
    "A": [["D", 20], ["F", 5]],
    "B": [["A", 5], ["E", 20]],
    "C": [["B", 5], ["F", 20]],
    "D": [["G", 20], ["I", 20]],
    "E": [["A", 5], ["D", 5], ["F", 5], ["G", 5], ["H", 20]],
    "F": [["B", 10], ["H", 20], ["I", 10]],
    "G": [["H", 5], ["T", 20]],
    "H": [["I", 5], ["T", 30]],
    "I": [["T", 25]],
    "T": []
}

In [9]:
ford_fulkerson_algorithm(flow_network_p2)

f: {'S': [['A', 0, 20], ['B', 0, 30], ['C', 0, 20]], 'A': [['D', 0, 20], ['F', 0, 5]], 'B': [['A', 0, 5], ['E', 0, 20]], 'C': [['B', 0, 5], ['F', 0, 20]], 'D': [['G', 0, 20], ['I', 0, 20]], 'E': [['A', 0, 5], ['D', 0, 5], ['F', 0, 5], ['G', 0, 5], ['H', 0, 20]], 'F': [['B', 0, 10], ['H', 0, 20], ['I', 0, 10]], 'G': [['H', 0, 5], ['T', 0, 20]], 'H': [['I', 0, 5], ['T', 0, 30]], 'I': [['T', 0, 25]], 'T': []}
r: {'S': [['A', 20], ['B', 30], ['C', 20]], 'A': [['D', 20], ['F', 5]], 'B': [['A', 5], ['E', 20]], 'C': [['B', 5], ['F', 20]], 'D': [['G', 20], ['I', 20]], 'E': [['A', 5], ['D', 5], ['F', 5], ['G', 5], ['H', 20]], 'F': [['B', 10], ['H', 20], ['I', 10]], 'G': [['H', 5], ['T', 20]], 'H': [['I', 5], ['T', 30]], 'I': [['T', 25]], 'T': []}
p: [['S', 'A'], ['A', 'D'], ['D', 'G'], ['G', 'T']]

f: {'S': [['A', 20, 20], ['B', 0, 30], ['C', 0, 20]], 'A': [['D', 20, 20], ['F', 0, 5]], 'B': [['A', 0, 5], ['E', 0, 20]], 'C': [['B', 0, 5], ['F', 0, 20]], 'D': [['G', 20, 20], ['I', 0, 20]], 'E': [

{'S': [['A', 20, 20], ['B', 25, 30], ['C', 20, 20]],
 'A': [['D', 20, 20], ['F', 5, 5]],
 'B': [['A', 5, 5], ['E', 20, 20]],
 'C': [['B', 0, 5], ['F', 20, 20]],
 'D': [['G', 20, 20], ['I', 0, 20]],
 'E': [['A', 0, 5], ['D', 0, 5], ['F', 0, 5], ['G', 0, 5], ['H', 20, 20]],
 'F': [['B', 0, 10], ['H', 15, 20], ['I', 10, 10]],
 'G': [['H', 0, 5], ['T', 20, 20]],
 'H': [['I', 5, 5], ['T', 30, 30]],
 'I': [['T', 15, 25]],
 'T': []}